In [7]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import model_selection as ms
pd.set_option("display.max_rows",15)
%matplotlib inline

In [46]:
no_of_lines = 383616 #train
train_size = int(no_of_lines * 0.01)
no_of_lines = 425995#test
test_size = int(no_of_lines * 0.1) 
print(train_size, test_size)

3836 42599


In [58]:
class dataset:
    col_names = ['sIP','dIP','sPort','dPort','pro','packets','bytes','flags','sTime','durat','eTime','sen','in','out','nhIP',
                 'cla','type','iTy','iCo','initialF','sessionF','attribut','appli','status',
                 'taxonomy','label','heuristic','distance','nbDetectors'
                ]

    sample_data = pd.read_csv("dataset/mawiitrain.csv", header = 0,names = col_names, nrows = train_size)
    sample_data.to_pickle('dataset/sample_train5.pkl')
    sample_data = pd.read_csv("dataset/20170827_mawilab_flow_006.csv", header = 0,names = col_names, nrows = test_size)
    sample_data.to_pickle('dataset/test51_file.pkl')

In [59]:
def process(df):
    min_port = np.minimum(df.sPort.values, df.dPort.values)
    df.insert(0,'svc_port',0)
    #df.loc[min_port <= 1024, 'svc_port'] = min_port 
    #df.loc[min_port > 1024, 'svc_port'] = df.dPort.copy()
    df = df.loc[:,('pro', 'packets', 'bytes', 'durat', 'status')].copy()
    return df
dataset.kdd_train = process(pd.read_pickle('dataset/sample_train5.pkl'))
dataset.kdd_test = process(pd.read_pickle('dataset/test51_file.pkl'))
dataset.kdd_label = dataset.kdd_test.status.copy()

dataset.kdd_test

,pro,packets,bytes,durat,status
0,6,1,52,0.0,anomaly
1,6,1,1500,0.0,anomaly
2,6,1,1420,0.0,anomaly
3,6,1,1500,0.0,anomaly
4,1,1,32,0.0,normal
5,6,1,1430,0.0,anomaly
6,6,1,1400,0.0,anomaly
...,...,...,...,...,...
42592,1,1,32,0.0,normal
42593,6,1,40,0.0,normal


In [60]:
dataset.kdd_label.unique()

array(['anomaly', 'normal'], dtype=object)

In [61]:
dataset.kdd_test.info(memory_usage = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42599 entries, 0 to 42598
Data columns (total 5 columns):
pro        42599 non-null int64
packets    42599 non-null int64
bytes      42599 non-null int64
durat      42599 non-null float64
status     42599 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 1.6+ MB


In [62]:
dataset.kdd_test.describe()

,pro,packets,bytes,durat
count,42599.000000,42599.0,42599.000000,42599.0
mean,5.839315,1.0,596.972840,0.0
std,7.243829,0.0,679.163401,0.0
min,1.000000,1.0,28.000000,0.0
25%,1.000000,1.0,32.000000,0.0
50%,6.000000,1.0,52.000000,0.0
75%,6.000000,1.0,1430.000000,0.0
max,50.000000,1.0,1500.000000,0.0


In [63]:
category_variables = ["status"]

for cv in category_variables:
    dataset.kdd_train[cv] = dataset.kdd_train[cv].astype("category")
    dataset.kdd_test[cv] = dataset.kdd_test[cv].astype("category", categories = dataset.kdd_train[cv].cat.categories)

c:\users\anusha\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """


In [64]:
dummy_variables_2labels = category_variables
    
class preprocessing:
    kdd_train_2labels = pd.get_dummies(dataset.kdd_train, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)
    kdd_test_2labels = pd.get_dummies(dataset.kdd_test, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)

In [65]:
preprocessing.kdd_train_2labels.to_pickle("dataset/preprocessed_train_data5.pkl")
preprocessing.kdd_test_2labels.to_pickle("dataset/preprocessed_test51_data.pkl")

preprocessing.kdd_test_2labels


,pro,packets,bytes,durat,status_anomaly,status_normal
0,6,1,52,0.0,1,0
1,6,1,1500,0.0,1,0
2,6,1,1420,0.0,1,0
3,6,1,1500,0.0,1,0
4,1,1,32,0.0,0,1
5,6,1,1430,0.0,1,0
6,6,1,1400,0.0,1,0
...,...,...,...,...,...,...
42592,1,1,32,0.0,0,1
42593,6,1,40,0.0,0,1


In [66]:
dataset.kdd_label.value_counts()

normal     29559
anomaly    13040
Name: status, dtype: int64

In [67]:
df_train = preprocessing.kdd_train_2labels.drop(['status_anomaly', 'status_normal'], axis = 1)
df_train = pd.concat([df_train, dataset.kdd_label], axis = 1)
df_train

,pro,packets,bytes,durat,status
0,1.0,1.0,32.0,0.0,anomaly
1,6.0,1.0,48.0,0.0,anomaly
2,6.0,1.0,1440.0,0.0,anomaly
3,6.0,1.0,52.0,0.0,anomaly
4,6.0,1.0,52.0,0.0,normal
5,6.0,1.0,40.0,0.0,anomaly
6,6.0,1.0,150.0,0.0,anomaly
...,...,...,...,...,...
42592,NaN,NaN,NaN,NaN,normal
42593,NaN,NaN,NaN,NaN,normal
